# Data Preparation

Focus on school years 2021-2022


In [1]:
# import libraries
import importlib
import os

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import subprocess
import sys

from pathlib import Path

In [2]:
# import other libraries
from helper import (
    load_cde_txt,
    clean_calschls_safety,
    clean_safety_by_connectedness,
    clean_columns,
)

# check if jcds library is installed
package_name = "jcds"

if importlib.util.find_spec(package_name) is None:
    print(f" '{package_name}' not found. Installing from Github... ")
    subprocess.check_call(
        [
            sys.executable,
            "-m",
            "pip",
            "install",
            "https://github.com/junclemente/jcds.git",
        ]
    )
else:
    print(f" '{package_name}' is already installed.")

from jcds import eda as jeda
from jcds import reports as jrep

 'jcds' is already installed.


In [3]:
data_folder = Path("../data")
ca_doe = Path(data_folder / "raw/ca_doe")
cde = Path(data_folder / "raw/cde")
ca_schls = Path(data_folder / "raw/ca_schls")
civil_rights = Path(data_folder / "raw/civil_rights_data")
raw_data = Path(data_folder / "raw")

# CA Dept of Education

## Adjusted Cohort Graduation Rate and Outcome Data (ACGR)

**Adjusted Cohort Graduation Rate and Outcome Data**
Four-year Adjusted Cohort Graduation Rate (ACGR) and Outcome data reported by race/ethnicity, student group, and gender.  
Source: [https://www.cde.ca.gov/ds/ad/filesacgr.asp](https://www.cde.ca.gov/ds/ad/filesacgr.asp)

**Note:** To protect student privacy, data are suppressed (\*) on the data file if the cell size within a selected student population (cohort students) is 10 or less. Additionally, the “Not Reported” race/ethnicity is suppressed, regardless of actual cell size, if the student population for one or more other race/ethnicity groups is suppressed.

Raw Datasets may not be included in the repository due to its size.
The datasets can be downloaded from here: [raw datasets](https://drive.google.com/drive/folders/1OUvpL1nY1uTu2PmjIHJ7wZmY4YgW4onM?usp=sharing)

[ACGR Data Dictionary](https://www.cde.ca.gov/ds/ad/fsacgr.asp)


In [4]:
# load raw dataset
df_raw = load_cde_txt(ca_doe / "acgr21.txt")

# filter dataset
df_acgr = df_raw[
    (df_raw["AggregateLevel"].str.strip() == "S")
    & (df_raw["CharterSchool"].str.strip() == "No")
    & (df_raw["DASS"].str.strip() == "No")
    & (df_raw["ReportingCategory"] == "TA")
]

df_acgr.to_pickle(data_folder / "raw_acgr.pkl")

| **Column Name**       | **Description**                                                                                                                                                                                                                                                           | **Values Kept** | **Reason for Filter**                                                                                                              |
| --------------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | --------------- | ---------------------------------------------------------------------------------------------------------------------------------- |
| **AggregateLevel**    | Indicates the data aggregation level within the ACGR dataset.<br><br>Possible values:<br>• `T` = State<br>• `C` = County<br>• `D` = District<br>• `S` = School                                                                                                            | `S` (School)    | Focuses analysis on **individual schools**, providing more data points and allowing finer-grained modeling of graduation outcomes. |
| **CharterSchool**     | Indicates whether the record includes data for charter schools.<br><br>Possible values:<br>• `All` = All schools combined<br>• `Y` = Charter only<br>• `N` = Non-charter only                                                                                             | `No` (or `N`)   | Excludes **charter schools** to focus on **traditional public high schools**.                                                      |
| **DASS**              | Dashboard Alternative School Status (DASS) flag.<br><br>Possible values:<br>• `All` = All schools combined<br>• `Y` = DASS only<br>• `N` = Non-DASS only                                                                                                                  | `No` (or `N`)   | Removes **alternative/continuation programs** so graduation rates reflect typical comprehensive high schools.                      |
| **ReportingCategory** | Identifies which subgroup the record represents.<br><br>Examples:<br>• Race/Ethnicity (`RB` = African American, `RH` = Hispanic, etc.)<br>• Gender (`GM` = Male, `GF` = Female)<br>• Program groups (`SD` = Students with Disabilities, etc.)<br>• `TA` = Total Aggregate | `TA`            | Keeps **aggregate totals** for each school (not broken down by subgroup) to simplify modeling.                                     |


In [5]:
cols_acgr = [
    "AcademicYear",
    "AggregateLevel",
    "CountyCode",
    "DistrictCode",
    "SchoolCode",
    "CountyName",
    "DistrictName",
    "SchoolName",
    "CharterSchool",
    "DASS",
    "ReportingCategory",
    "CohortStudents",  # QA for weighing
    "Regular HS Diploma Graduates (Rate)",  # target
    "Met UC/CSU Grad Req's (Rate)",  # academic readiness/intensity feature
    "Seal of Biliteracy (Rate)",  # ??? language proficiency proxy
    "Golden State Seal Merit Diploma (Rate",  # ???
    "Dropout (Rate)",  # 2nd target
    "Still Enrolled (Rate)",  # 5th year senior
]

# CA Dept of Education

[Downloadable Data Files](https://www.cde.ca.gov/ds/ad/downloadabledata.asp)  
Downloadable files about California's K–12 educational system by topic area, including enrollment, assessment and accountability, English learners, foster youth, free or reduced-price meal, graduates and dropouts, and staff data.


## Chronic Absenteeism Data

The Absenteeism Downloadable Files page provides access to data about student absenteeism, including chronic absenteeism and absenteeism by reason counts and rates, disaggregated by race/ethnicity, gender, student program group, and grade span.

[Chronic Absenteeism Data Dictionary](https://www.cde.ca.gov/ds/ad/fsabd.asp)


In [6]:
# load raw dataset and filter
df_raw = load_cde_txt(cde / "chronicabsenteeism21.txt")

df_chron_abs = df_raw[
    (df_raw["Aggregate Level"].str.strip() == "S")
    & (df_raw["Charter School"].str.strip() == "No")
    & (df_raw["Reporting Category"].str.strip() == "TA")
]

# save
df_chron_abs.to_pickle(data_folder / "raw_chronic_absent.pkl")

In [7]:
chron_abs_cols = [
    "County Code",
    "District Code",
    "School Code",
    "County Name",
    "District Name",
    "School Name",
    "ChronicAbsenteeismRate",
]

df_chron_abs[chron_abs_cols]

,County Code,District Code,School Code,County Name,District Name,School Name,ChronicAbsenteeismRate
57598,01,10017,0130419,Alameda,Alameda County Office of Education,Alameda County Community,84.4
57599,01,10017,0130401,Alameda,Alameda County Office of Education,Alameda County Juvenile Hall/Court,61.7
57621,01,31609,0131755,Alameda,California School for the Blind (State Special...,California School for the Blind,8.8
57644,01,31617,0131763,Alameda,California School for the Deaf-Fremont (State ...,California School for the Deaf-Fremont,11.6
58027,01,61119,6090013,Alameda,Alameda Unified,Edison Elementary,3.7
...,...,...,...,...,...,...,...
263006,58,72751,6056832,Yuba,Wheatland,Lone Tree Elementary,16.7
263008,58,72751,6056840,Yuba,Wheatland,Wheatland Elementary,27.1
263062,58,72769,0123570,Yuba,Wheatland Union High,Wheatland Community Day High,NaN
263063,58,72769,0133751,Yuba,Wheatland Union High,Edward P. Duplex,100


### Absenteeism by Reason

File structure for the Absenteeism by Reason data reported by race/ethnicity, gender, student group, and grade span. "Eligible" cumulative enrollment, students with one or more absence, average days absent, and absences by reason are provided.

[Absenteeism by Reeason Data Dictionary](https://www.cde.ca.gov/ds/ad/fsabr.asp)


In [8]:
df_raw = load_cde_txt(cde / "absenteeismreason22-v3.txt")
df_raw

df_abs = df_raw[
    (df_raw["Aggregate Level"].str.strip() == "S")
    & (df_raw["Charter School"].str.strip() == "No")
    & (df_raw["DASS"].str.strip() == "No")
    & (df_raw["Reporting Category"] == "TA")
]

df_abs.to_pickle(data_folder / "raw_absent_reason.pkl")

In [9]:
abs_cols = [
    "District Code",
    "School Code",
    "County Name",
    "District Name",
    "School Name",
    "Unexcused Absences (percent)",
    "Out-of-School Suspension Absences (percent)",
]
df_abs[abs_cols]

,District Code,School Code,County Name,District Name,School Name,Unexcused Absences (percent),Out-of-School Suspension Absences (percent)
583,31609,0131755,Alameda,California School for the Blind (State Special...,California School for the Blind,46.9,0
608,31617,0131763,Alameda,California School for the Deaf-Fremont (State ...,California School for the Deaf-Fremont,41.8,1.8
628,61119,0000000,Alameda,Alameda Unified,District Office,0,0
647,61119,0106401,Alameda,Alameda Unified,Alameda Science and Technology Institute,20.2,0
670,61119,0111765,Alameda,Alameda Unified,Ruby Bridges Elementary,28.4,0.2
...,...,...,...,...,...,...,...
227659,72751,6056816,Yuba,Wheatland,Bear River,43.9,0.2
227682,72751,6056832,Yuba,Wheatland,Lone Tree Elementary,29,0.1
227706,72751,6056840,Yuba,Wheatland,Wheatland Elementary,33.5,0
227743,72769,0000000,Yuba,Wheatland Union High,District Office,*,*


## Public Schools and Districts

The Public Schools and Districts Downloadable Files page provides access to data files containing general information about California's public schools and districts found in the California School Directory.

[Public Schools and Districts](https://www.cde.ca.gov/ds/si/ds/pubschls.asp)

[Data Dictionary](https://www.cde.ca.gov/ds/si/ds/fspubschls.asp)


In [10]:
df_raw = pd.read_excel(cde / "pubschls.xlsx", header=5)

df_schooldata = df_raw[
    (df_raw["StatusType"].str.strip() == "Active")
    & (df_raw["EdOpsCode"].str.strip() == "TRAD")
]

df_schooldata.to_pickle(data_folder / "raw_school_data.pkl")

In [11]:
cols_schooldata = [
    "CDSCode",
    "NCESDist",
    "NCESSchool",
    "StatusType",
    "County",
    "District",
    "School",
    "Zip",
    "OpenDate",
    "Charter",
    "DOCType",
    "SOCType",
    "EdOpsCode",
    "EILCode",
    "GSserved",
    "Virtual",
    "Magnet",
    "YearRoundYN",
    "FederalDFCDistrictID",
    "Latitude",
    "Longitude",
    "LastUpDate",
    "Multilingual",
]

df_schooldata[cols_schooldata]

,CDSCode,NCESDist,NCESSchool,StatusType,County,District,School,Zip,OpenDate,Charter,...,EILCode,GSserved,Virtual,Magnet,YearRoundYN,FederalDFCDistrictID,Latitude,Longitude,LastUpDate,Multilingual
2,01100170112607,0691051,10947,Active,Alameda,Alameda County Office of Education,Envision Academy for Arts & Technology,94612-3355,2006-08-28 00:00:00,Y,...,ELEMHIGH,6-12,N,N,N,0601614,37.804520,-122.26815,2024-05-20,N
3,01100170114363,0691051,12013,Active,Alameda,Alameda County Office of Education,American Indian Public Charter School II,94607-4900,2007-07-01 00:00:00,Y,...,ELEM,K-8,N,N,N,0601880,37.800368,-122.26548,2025-09-08,N
5,01100170123968,0691051,12844,Active,Alameda,Alameda County Office of Education,Community School for Creative Education,94606-4903,2011-08-22 00:00:00,Y,...,ELEM,K-8,N,N,N,0601691,37.784648,-122.23863,2024-07-30,Y
6,01100170124172,0691051,12901,Active,Alameda,Alameda County Office of Education,Yu Ming Charter,94607-2477,2011-08-09 00:00:00,Y,...,ELEM,K-8,N,N,N,0602013,37.818228,-122.28233,2024-05-15,Y
8,01100170126748,0691051,13155,Active,Alameda,Alameda County Office of Education,LPS Oakland R & D Campus,94605-4037,2012-08-21 00:00:00,Y,...,HS,9-12,N,N,N,0601967,37.759536,-122.16291,2025-07-01,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18390,58727516056816,0642330,06925,Active,Yuba,Wheatland,Bear River,95692-9286,1980-07-01 00:00:00,N,...,INTMIDJR,4-8,N,N,N,No Data,39.007843,-121.43617,2023-02-09,N
18392,58727516056832,0642330,06927,Active,Yuba,Wheatland,Lone Tree Elementary,95903,1980-07-01 00:00:00,N,...,ELEM,K-5,N,N,N,No Data,39.100027,-121.33614,2023-02-09,N
18393,58727516056840,0642330,06928,Active,Yuba,Wheatland,Wheatland Elementary,95692-8215,1980-07-01 00:00:00,N,...,ELEM,K-3,N,N,N,No Data,39.012487,-121.42900,2024-08-15,N
18396,58727516118806,0642330,11548,Active,Yuba,Wheatland,Wheatland Charter Academy,95903,2001-08-22 00:00:00,Y,...,ELEM,K-5,N,N,N,No Data,39.102475,-121.33536,2025-07-02,N


## Free or Reduced-Price Meal (Student Poverty)

The Free or Reduced-Price Meal Downloadable Files page provides access to data about students who are eligible for Free or Reduced-Price Meals (FRPM).

[FRPM Data Dictionary](https://www.cde.ca.gov/ds/ad/fsspfrpm.asp)


In [12]:
df_raw = pd.read_excel(
    cde / "frpm2122_v2.xlsx", sheet_name="FRPM School-Level Data ", header=1
)

df_raw = clean_columns(df_raw)

df_frpm = df_raw[df_raw["Charter School (Y/N)"].str.strip() == "N"]

df_frpm.to_pickle(data_folder / "raw_frpm.pkl")

In [13]:
cols_frpm = [
    "Academic Year",
    "County Code",
    "District Code",
    "School Code",
    "County Name",
    "District Name",
    "School Name",
    "District Type",
    "School Type",
    "Educational Option Type",
    "Charter School (Y/N)",
    "IRC",
    "Enrollment (K-12)",
    "Percent (%) Eligible Free (K-12)",
    "FRPM Count (K-12)",
    "Percent (%) Eligible FRPM (K-12)",
    "CALPADS Fall 1 Certification Status",
]

df_frpm[cols_frpm]

,Academic Year,County Code,District Code,School Code,County Name,District Name,School Name,District Type,School Type,Educational Option Type,Charter School (Y/N),IRC,Enrollment (K-12),Percent (%) Eligible Free (K-12),FRPM Count (K-12),Percent (%) Eligible FRPM (K-12),CALPADS Fall 1 Certification Status
0,2021-2022,1,10017,130419,Alameda,Alameda County Office of Education,Alameda County Community,County Office of Education (COE),County Community,County Community School,N,N,57,0.789474,47,0.824561,Y
1,2021-2022,1,10017,130401,Alameda,Alameda County Office of Education,Alameda County Juvenile Hall/Court,County Office of Education (COE),Juvenile Court Schools,Juvenile Court School,N,N,64,1.000000,64,1.000000,Y
14,2021-2022,1,31609,131755,Alameda,California School for the Blind (State Special...,California School for the Blind,State Special Schools,State Special Schools,State Special School,N,N,62,1.000000,62,1.000000,Y
15,2021-2022,1,31617,131763,Alameda,California School for the Deaf-Fremont (State ...,California School for the Deaf-Fremont,State Special Schools,State Special Schools,State Special School,N,N,318,1.000000,318,1.000000,Y
17,2021-2022,1,61119,130229,Alameda,Alameda Unified,Alameda High,Unified School District,High Schools (Public),Traditional,N,N,1808,0.172013,327,0.180863,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10550,2021-2022,58,72751,6056832,Yuba,Wheatland,Lone Tree Elementary,Elementary School District,Elementary Schools (Public),Traditional,N,N,351,0.193732,119,0.339031,Y
10552,2021-2022,58,72751,6056840,Yuba,Wheatland,Wheatland Elementary,Elementary School District,Elementary Schools (Public),Traditional,N,N,340,0.523529,189,0.555882,Y
10554,2021-2022,58,72769,133751,Yuba,Wheatland Union High,Edward P. Duplex,High School District,Continuation High Schools,Continuation School,N,N,45,0.711111,45,1.000000,Y
10556,2021-2022,58,72769,123570,Yuba,Wheatland Union High,Wheatland Community Day High,High School District,District Community Day Schools,Community Day School,N,N,5,0.600000,4,0.800000,Y


## CBEDS Data about Schools & Districts

Downloadable data files for information about schools and districts, including estimated number of teacher hires, work visa applications, home-to-school transportation, kindergarten program type, and educational calendar.  
[CBEDS Data Dictionary](https://www.cde.ca.gov/ds/ad/fscbedsorab19.asp)


In [14]:
df_raw = load_cde_txt(cde / "cbedsora21b.txt")

df_cbeds = df_raw[df_raw["Level"].str.strip() == "S"]

df_cbeds.to_pickle(data_folder / "raw_cbeds.pkl")

In [15]:
col_cbeds = [
    "Cdscode",
    "CountyName",
    "DistrictName",
    "SchoolName",
    "Level",
    "Value",
    "Year",
]
df_cbeds[col_cbeds]

,Cdscode,CountyName,DistrictName,SchoolName,Level,Value,Year
18,01100170112607,Alameda,Alameda County Office of Education,Envision Academy for Arts & Technology,S,True,2122
19,01100170112607,Alameda,Alameda County Office of Education,Envision Academy for Arts & Technology,S,True,2122
20,01100170112607,Alameda,Alameda County Office of Education,Envision Academy for Arts & Technology,S,0,2122
21,01100170112607,Alameda,Alameda County Office of Education,Envision Academy for Arts & Technology,S,0,2122
22,01100170112607,Alameda,Alameda County Office of Education,Envision Academy for Arts & Technology,S,True,2122
...,...,...,...,...,...,...,...
58706,58727695838305,Yuba,Wheatland Union High,Wheatland Union High,S,True,2122
58707,58727695838305,Yuba,Wheatland Union High,Wheatland Union High,S,True,2122
58708,58727695838305,Yuba,Wheatland Union High,Wheatland Union High,S,True,2122
58709,58727695838305,Yuba,Wheatland Union High,Wheatland Union High,S,20210811,2122


## Staff Data Files

The Staff Downloadable Files page provides access to data about certificated and classified staff demographic information, staff assignments, student/staff ratios, and estimated teacher hires.

### Student / Staff Ratio

[Student-Staff Ratio Data Dictionary](https://www.cde.ca.gov/ds/ad/fsstrat.asp)


In [16]:
df_raw = load_cde_txt(cde / "strat2122.txt")


df_ss_ratio = df_raw[
    (df_raw["Aggregate Level"].str.strip() == "S")
    & (df_raw["Charter School"].str.strip() == "N")
    & (df_raw["DASS"].str.strip() == "N")
    # & (df_raw["Reporting Category"] == "TA")
]

df_ss_ratio.to_pickle(data_folder / "raw_student_staff_ratio.pkl")

In [17]:
cols_ss_ratio = [
    "Academic Year",
    "Aggregate Level",
    "County Code",
    "District Code",
    "School Code",
    "County Name",
    "District Name",
    "School Name",
    "Charter School",
    "DASS",
    "School Grade Span",
    "STU_TCH_RATIO",  # student / teacher ratio
    "STU_ADM_RATIO",  # student / admin ratio
    "STU_PSV_RATIO",  # student / counselor ratio
]

df_ss_ratio[cols_ss_ratio]

,Academic Year,Aggregate Level,County Code,District Code,School Code,County Name,District Name,School Name,Charter School,DASS,School Grade Span,STU_TCH_RATIO,STU_ADM_RATIO,STU_PSV_RATIO
556,2021-22,S,01,10017,0000000,Alameda,Alameda County Office of Education,District Office,N,N,GS_K12,*,*,*
571,2021-22,S,01,31609,0131755,Alameda,California School for the Blind (State Special...,California School for the Blind,N,N,GS_K12,4.8,12.4,3.9
572,2021-22,S,01,31617,0000000,Alameda,California School for the Deaf-Fremont (State ...,District Office,N,N,GS_K12,*,*,*
573,2021-22,S,01,31617,0131763,Alameda,California School for the Deaf-Fremont (State ...,California School for the Deaf-Fremont,N,N,GS_K12,4.4,40.3,159
574,2021-22,S,01,61119,0000000,Alameda,Alameda Unified,District Office,N,N,GS_K12,*,*,*
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30235,2021-22,S,58,72751,6056816,Yuba,Wheatland,Bear River,N,N,GS_K12,18.9,284,180.3
30236,2021-22,S,58,72751,6056832,Yuba,Wheatland,Lone Tree Elementary,N,N,GS_K6,20.6,*,175.5
30237,2021-22,S,58,72751,6056840,Yuba,Wheatland,Wheatland Elementary,N,N,GS_K6,20,340,123.6
30239,2021-22,S,58,72769,0000000,Yuba,Wheatland Union High,District Office,N,N,GS_K12,*,3,*


In [18]:
df_ss_ratio.columns.to_list()

['Academic Year',
 'Aggregate Level',
 'County Code',
 'District Code',
 'School Code',
 'County Name',
 'District Name',
 'School Name',
 'Charter School',
 'DASS',
 'School Grade Span',
 'TOTAL_ENR_N',
 'TCH_FTE_N',
 'ADM_FTE_N',
 'PSV_FTE_N',
 'OTH_FTE_N',
 'STU_TCH_RATIO',
 'STU_ADM_RATIO',
 'STU_PSV_RATIO',
 'STU_OTH_RATIO']

### Staff Education

[Staff Education Data Dictionary](https://www.cde.ca.gov/ds/ad/fssted.asp)


In [19]:
df_raw = load_cde_txt(cde / "sted2122.txt")

df_staff_ed = df_raw[
    (df_raw["Aggregate Level"].str.strip() == "S")
    & (df_raw["Charter School"].str.strip() == "N")
    & (df_raw["DASS"].str.strip() == "N")
    # & (df_raw["Reporting Category"] == "TA")
]

df_staff_ed.to_pickle(data_folder / "raw_staff_edu.pkl")

In [20]:
cols_staff_ed = [
    "Academic Year",
    "Aggregate Level",
    "County Code",
    "District Code",
    "School Code",
    "County Name",
    "District Name",
    "School Name",
    "Charter School",
    "DASS",
    "Staff Type",
    "School Grade Span",
    "Total Staff Count",
    "Associate",
    "Baccalaureate",
    "Baccalaureate Plus",
    "Master",
    "Master Plus",
    "Doctorate",
    "Special (Juris Doctor)",
    "None",
]
df_staff_ed[cols_staff_ed]

,Academic Year,Aggregate Level,County Code,District Code,School Code,County Name,District Name,School Name,Charter School,DASS,...,School Grade Span,Total Staff Count,Associate,Baccalaureate,Baccalaureate Plus,Master,Master Plus,Doctorate,Special (Juris Doctor),None
7395,2021-22,S,01,10017,0000000,Alameda,Alameda County Office of Education,District Office,N,N,...,GS_K12,5,0,0,0,0,4,1,0,0
7396,2021-22,S,01,10017,0000000,Alameda,Alameda County Office of Education,District Office,N,N,...,GS_K12,5,0,0,0,0,4,1,0,0
7397,2021-22,S,01,10017,0000000,Alameda,Alameda County Office of Education,District Office,N,N,...,GS_K12,6,0,0,0,0,5,1,0,0
7398,2021-22,S,01,10017,0000000,Alameda,Alameda County Office of Education,District Office,N,N,...,GS_K12,6,0,0,0,0,5,1,0,0
7399,2021-22,S,01,10017,0000000,Alameda,Alameda County Office of Education,District Office,N,N,...,GS_K12,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360896,2021-22,S,58,10587,0000000,Yuba,Yuba County Office of Education,District Office,N,N,...,GS_K12,11,0,2,1,7,0,1,0,0
360897,2021-22,S,58,10587,0000000,Yuba,Yuba County Office of Education,District Office,N,N,...,GS_K12,9,0,2,1,6,0,0,0,0
360898,2021-22,S,58,10587,0000000,Yuba,Yuba County Office of Education,District Office,N,N,...,GS_K12,2,0,0,0,1,0,1,0,0
360899,2021-22,S,58,10587,0000000,Yuba,Yuba County Office of Education,District Office,N,N,...,GS_K12,1,0,0,1,0,0,0,0,0


### Staff Experience

[Staff Experience Data Dictionary](https://www.cde.ca.gov/ds/ad/fsstex.asp)


In [21]:
df_raw = load_cde_txt(cde / "stex2122.txt")

df_staff_xp = df_raw[
    (df_raw["Aggregate Level"].str.strip() == "S")
    & (df_raw["Charter School"].str.strip() == "N")
    & (df_raw["DASS"].str.strip() == "N")
]

df_staff_xp.to_pickle(data_folder / "raw_staff_exp.pkl")

In [22]:
cols_staff_xp = [
    "Academic Year",
    "Aggregate Level",
    "County Code",
    "District Code",
    "School Code",
    "County Name",
    "District Name",
    "School Name",
    "Charter School",
    "DASS",
    "Staff Type",
    "School Grade Span",
    "Total Staff Count",
    "Average Total Years Experience",
    "Average District Years Experience",
    "Experienced",  # 2+ years experience
    "Inexperienced",  # <2 years experience
    "First Year",  # No.of staff in 1st year
    "Second Year",  # No. of staff in 2nd year
]
df_staff_xp

,Academic Year,Aggregate Level,County Code,District Code,School Code,County Name,District Name,School Name,Charter School,DASS,Staff Type,School Grade Span,Staff Gender,Total Staff Count,Average Total Years Experience,Average District Years Experience,Experienced,Inexperienced,First Year,Second Year
7395,2021-22,S,01,10017,0000000,Alameda,Alameda County Office of Education,District Office,N,N,ADM,GS_K12,ALL,5,20.6,9.2,5,0,0,0
7396,2021-22,S,01,10017,0000000,Alameda,Alameda County Office of Education,District Office,N,N,ADM,GS_K12,GF,5,20.6,9.2,5,0,0,0
7397,2021-22,S,01,10017,0000000,Alameda,Alameda County Office of Education,District Office,N,N,ALL,GS_K12,ALL,6,20.0,10.2,6,0,0,0
7398,2021-22,S,01,10017,0000000,Alameda,Alameda County Office of Education,District Office,N,N,ALL,GS_K12,GF,6,20.0,10.2,6,0,0,0
7399,2021-22,S,01,10017,0000000,Alameda,Alameda County Office of Education,District Office,N,N,OTH,GS_K12,ALL,1,17.0,15.0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360896,2021-22,S,58,10587,0000000,Yuba,Yuba County Office of Education,District Office,N,N,ALL,GS_K12,ALL,11,21.6,14.6,11,0,0,0
360897,2021-22,S,58,10587,0000000,Yuba,Yuba County Office of Education,District Office,N,N,ALL,GS_K12,GF,9,22.8,16.0,9,0,0,0
360898,2021-22,S,58,10587,0000000,Yuba,Yuba County Office of Education,District Office,N,N,ALL,GS_K12,GM,2,16.5,8.5,2,0,0,0
360899,2021-22,S,58,10587,0000000,Yuba,Yuba County Office of Education,District Office,N,N,OTH,GS_K12,ALL,1,16.0,15.0,1,0,0,0


### Enrollment by School

[Enrollment by School Data Dictionary](https://www.cde.ca.gov/ds/ad/fsenrps.asp)


In [23]:
df_raw = load_cde_txt(cde / "enr202022-v2.txt")

df_enroll = df_raw[df_raw["ENR_TYPE"] == "P"]

df_enroll.to_pickle(data_folder / "raw_school_enroll.pkl")

In [24]:
cols_enroll = [
    "ACADEMIC_YEAR",
    "CDS_CODE",
    "COUNTY",
    "DISTRICT",
    "SCHOOL",
    "ENR_TYPE",
    "RACE_ETHNICITY",
    "GENDER",
    "GR_9",
    "GR_10",
    "GR_11",
    "GR_12",
    "UNGR_SEC",
    "ENR_TOTAL",
]

df_enroll[cols_enroll]

,ACADEMIC_YEAR,CDS_CODE,COUNTY,DISTRICT,SCHOOL,ENR_TYPE,RACE_ETHNICITY,GENDER,GR_9,GR_10,GR_11,GR_12,UNGR_SEC,ENR_TOTAL
15,2020-21,01100170112607,ALAMEDA,Alameda County Office of Education,Envision Academy for Arts & Technology,P,0,F,1,0,1,0,0,6
16,2020-21,01100170112607,ALAMEDA,Alameda County Office of Education,Envision Academy for Arts & Technology,P,0,M,0,1,1,0,0,3
17,2020-21,01100170112607,ALAMEDA,Alameda County Office of Education,Envision Academy for Arts & Technology,P,1,F,1,1,0,1,0,3
18,2020-21,01100170112607,ALAMEDA,Alameda County Office of Education,Envision Academy for Arts & Technology,P,2,F,1,0,0,0,0,1
19,2020-21,01100170112607,ALAMEDA,Alameda County Office of Education,Envision Academy for Arts & Technology,P,2,M,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
793375,2022-23,58727695838305,YUBA,Wheatland Union High,Wheatland Union High,P,7,F,35,54,60,43,0,192
793376,2022-23,58727695838305,YUBA,Wheatland Union High,Wheatland Union High,P,7,M,61,67,61,63,0,252
793377,2022-23,58727695838305,YUBA,Wheatland Union High,Wheatland Union High,P,7,X,0,0,1,0,0,1
793378,2022-23,58727695838305,YUBA,Wheatland Union High,Wheatland Union High,P,9,F,16,10,14,12,0,52


## Ca DOE School Climate, Health, and Learning Surveys

### Perception of Safety by Grade Level

[https://calschls.org/reports-data/query-calschls/?ind=58](https://calschls.org/reports-data/query-calschls/?ind=58)

The safety and supportiveness of young people's school environments play a crucial role in their development and academic success. Students who feel safe and supported at school tend to have better emotional health and are less likely to engage in risky behaviors (1, 2). Exposure to violence in schools and school neighborhoods is associated with many negative outcomes for youth, including poor academic performance, truancy, substance use, violent behavior, depression-related feelings, and suicidal thoughts and behaviors (1, 3). Experiencing violence during childhood or adolescence also increases the likelihood of long-term physical, behavioral, and mental health problems in adulthood (1). Further, school violence not only affects the individuals involved but also can adversely impact teachers, bystanders, and surrounding communities (3).

Unfortunately, school safety is often compromised. According to a 2019 survey, nearly half (44%) of U.S. high school students had one or more violent experiences in the previous year, such as bullying, physical fighting, being threatened with a weapon at school, dating violence, or sexual violence (1). Females and LGBTQ students were significantly more likely to experience multiple types of violence when compared with males and heterosexual students, respectively (1). In addition, studies show that reports of hate crimes and mass casualty events in schools have increased in recent years (3, 4).


In [25]:
df_raw = pd.read_excel(
    ca_schls / "Kidsdata-Perceptions-of-School-Safety--by-Grade-Level--2017.xls",
    header=None,
)

df_safety = clean_calschls_safety(df_raw)

df_safety.to_pickle(data_folder / "raw_safety_percept_grade.pkl")

### Perception of Safety by School Connectedness

[https://calschls.org/reports-data/query-calschls/?ind=60](https://calschls.org/reports-data/query-calschls/?ind=60)

The safety and supportiveness of young people's school environments play a crucial role in their development and academic success. Students who feel safe and supported at school tend to have better emotional health and are less likely to engage in risky behaviors (1, 2). Exposure to violence in schools and school neighborhoods is associated with many negative outcomes for youth, including poor academic performance, truancy, substance use, violent behavior, depression-related feelings, and suicidal thoughts and behaviors (1, 3). Experiencing violence during childhood or adolescence also increases the likelihood of long-term physical, behavioral, and mental health problems in adulthood (1). Further, school violence not only affects the individuals involved but also can adversely impact teachers, bystanders, and surrounding communities (3).

Unfortunately, school safety is often compromised. According to a 2019 survey, nearly half (44%) of U.S. high school students had one or more violent experiences in the previous year, such as bullying, physical fighting, being threatened with a weapon at school, dating violence, or sexual violence (1). Females and LGBTQ students were significantly more likely to experience multiple types of violence when compared with males and heterosexual students, respectively (1). In addition, studies show that reports of hate crimes and mass casualty events in schools have increased in recent years (3, 4).


In [26]:
df_raw = pd.read_excel(
    ca_schls / "Kidsdata-Perceptions-of-School-Safety--by-Level-of-School-C.xls",
    header=None,
)

df_connected = clean_safety_by_connectedness(df_raw)

df_connected.to_pickle(data_folder / "raw_safety_connect.pkl")